# ROA vs. P/E Investment Evaluation

### Overarching Principle: 
> Finding stocks that are undervalued.

To do so, we refer to the method used by Joel Greenblat in his book The Little Book That Beats the Market. 
We will use ***Return on Asset*** (ROA) to reflect how profitable a company is, and we will use ***Trailing Price-To-Earnings*** (Trailing PE) to find out if a stock is a relative bargain compare to other stocks. 

Note: This method cannot identify growth stock, since it does not take into considertaion of a company's future. 





In [ ]:
# !pip install fix-yahoo-finance==0.1.3
!pip install yfinance==0.1.70

In [ ]:
import yfinance as yf   # for getting information from yahoo finance
# yfinance use reference: https://towardsdatascience.com/financial-data-from-yahoo-finance-with-python-b5399743bcc6
import pandas as pd
import seaborn as sns
import plotly.express as px

In [ ]:
from pandas.core.apply import Apply
# test getting data
tick = yf.Ticker('MRNA')
print(tick.info)
print(tick.info['open'])
print(tick.info.get('trailingPE'))
print(tick.info['trailingEps'])
print(tick.info['returnOnAssets'])
# print(tick.info['open']/tick.info['trailingEps'])

{'zip': '02139', 'sector': 'Healthcare', 'fullTimeEmployees': 3000, 'longBusinessSummary': 'Moderna, Inc., a biotechnology company, develops therapeutics and vaccines based on messenger RNA for the treatment of infectious diseases, immuno-oncology, rare diseases, cardiovascular diseases, and auto-immune diseases. As of March 9, 2021, the company had 13 programs in clinical trials and a total of 24 development programs in six modalities comprising prophylactic vaccines, cancer vaccines, intratumoral immuno-oncology, localized regenerative therapeutics, systemic secreted and cell surface therapeutics, and systemic intracellular therapeutics. The company has strategic alliances with AstraZeneca PLC, Merck & Co., Inc., Vertex Pharmaceuticals Incorporated, Vertex Pharmaceuticals (Europe) Limited, the Biomedical Advanced Research and Development Authority, the Defense Advanced Research Projects Agency, the National Institute of Allergy and Infectious Diseases, the National Institutes of Heal

## Start the model building

In [ ]:
# get list of stocks wished to compare
# ticker_list = ['SBUX', 'F', 'OXY']
ticker_list = ['SBUX', 'OXY', 'AAPL', 'MSFT', 'TSLA', 'DIS', 'AMD', 'GOOGL', 'QCOM', 'SONY', 'PLUG', 'TGT', 'MCD', 
               'INTC', 'BAC', 'BA', 'F', 'AMZN', 'NFLX', 'HPQ', 'BABA', 'DAL', 'GS', 'JPM', 'WMT', 'FB', 'UAA',
               'PLNT', 'BILI', 'JBLU', 'XOM', 'JNJ']

In [ ]:
# create empty dataframe
df = pd.DataFrame(columns = ['Ticker', 'ROA', 'TrailingPE'])
print(df)

Empty DataFrame
Columns: [Ticker, ROA, TrailingPE]
Index: []


In [ ]:
from numpy import NaN
for ticker in ticker_list:
  # adding ticker object
  tick_info = yf.Ticker(ticker)
  # getting ROA
  ROA = tick_info.info.get('returnOnAssets')
  # getting trailing PE
  try:
    trailingPE = tick_info.info.get('trailingPE')
  except KeyError:
    print('Trailing PE not found for', ticker + '.', 'Calculating using opening price divided by trailing EPS')
    trailingPE = tick_info.info.get('open')/tick_info.info.get('trailingEps')
  if trailingPE is None:
    trailingPE = NaN
    print('Trailing PE of', ticker, 'is less than 0, use NaN as value')
  # create dictionary for this ticker
  ticker_info = pd.DataFrame({'Ticker': [ticker], 'ROA': [ROA], 'TrailingPE': [trailingPE]})
  # update dataframe
  df = df.append(ticker_info)

Trailing PE of PLUG is less than 0, use NaN as value
Trailing PE of BA is less than 0, use NaN as value
Trailing PE of DAL is less than 0, use NaN as value
Trailing PE of BILI is less than 0, use NaN as value
Trailing PE of JBLU is less than 0, use NaN as value
Trailing PE of XOM is less than 0, use NaN as value


In [ ]:
df.reset_index(drop=True, inplace = True)
df.head()

,Ticker,ROA,TrailingPE
0,SBUX,0.10477,25.014862
1,OXY,0.04048,24.518988
2,AAPL,0.19875,27.406485
3,MSFT,0.15249,31.665780
4,TSLA,0.07135,165.245860


## Add rank to dataframe

In [ ]:
df['ROA Rank'] = df['ROA'].rank(ascending = False)
df['Trailing PE Rank'] = df['TrailingPE'].rank(na_option = 'bottom')
df['Value Points'] = df['ROA Rank'] + df['Trailing PE Rank']
df['Overall Rank'] = df['Value Points'].rank()
df

,Ticker,ROA,TrailingPE,ROA Rank,Trailing PE Rank,Value Points,Overall Rank
0,SBUX,0.10477,25.014862,8.0,17.0,25.0,12.0
1,OXY,0.04048,24.518988,21.0,14.0,35.0,20.5
2,AAPL,0.19875,27.406485,2.0,18.0,20.0,7.0
3,MSFT,0.15249,31.665780,5.0,20.0,25.0,12.0
4,TSLA,0.07135,165.245860,14.0,26.0,40.0,23.5
5,DIS,0.01690,88.847300,25.0,24.0,49.0,27.0
6,AMD,0.21327,47.105057,1.0,22.0,23.0,9.5
7,GOOGL,0.14493,23.968466,6.0,13.0,19.0,6.0
8,QCOM,0.17321,19.588585,4.0,10.0,14.0,3.0
9,SONY,0.02392,15.291611,23.0,9.0,32.0,18.5


## Visualize ROA and PE

In [ ]:
# use plotly to create interactive scatterplot for ROA vs Trailing PE
# https://plotly.com/python/line-and-scatter/
fig = px.scatter(df, x = 'ROA', y = 'TrailingPE', color = 'Ticker', text = 'Ticker', title = 'ROA vs Trailing PE Plot')
fig.update_traces(marker_size = 15)
fig.show()

In [ ]:
# plot ROA rank
df_roa = df.sort_values(by='ROA Rank', ascending = False)
px.bar(df_roa, x = 'ROA Rank', y = 'Ticker', orientation = 'h', title = 'Ranking by ROA', hover_data=["ROA"])

In [ ]:
# plot Trailing PE rank
df_roa = df.sort_values(by='Trailing PE Rank', ascending = False)
px.bar(df_roa, x = 'Trailing PE Rank', y = 'Ticker', orientation = 'h', title = 'Ranking by Trailing PE', hover_data=["TrailingPE"])

In [ ]:
# plot Trailing overall rank
df_roa = df.sort_values(by='Overall Rank', ascending = False)
px.bar(df_roa, x = 'Overall Rank', y = 'Ticker', orientation = 'h', title = 'Overall Ranking')